In [1]:
import requests
from fake_useragent import UserAgent
import re
import pandas as pd
from io import BytesIO
import datetime

In [47]:
ticker = 'AAPL'

In [48]:
ua = UserAgent()
headers = {'User-Agent': ua.chrome}

In [49]:
r = requests.get('https://finance.yahoo.com/quote/SPY/history?p={}'.format(ticker), headers=headers)

In [50]:
r

<Response [200]>

In [51]:
r.text

'<!DOCTYPE html><html id="atomic" class="NoJs chrome desktop" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charset="utf-8"/><title>SPY Historical Prices | SPDR S&amp;P 500 Stock - Yahoo Finance</title><meta name="keywords" content="SPY, SPDR S&amp;P 500, SPY historical prices, SPDR S&amp;P 500 historical prices, historical prices, stocks, quotes, finance"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="90376669494"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="en-US" content="Discover historical prices for SPY stock on Yahoo Finance. View daily, weekly or monthly format back to when SPDR S&amp;P 500 stock was issued."/><link rel="dns-prefetch" href="//l.yimg.com"/><link rel="dns-prefetch"

In [52]:
crumb_regex = r'"CrumbStore":{"crumb":"(.*?)"}'
crumb = re.search(crumb_regex, str(r.content)).group(1)
crumb

'bcBshTq.f33'

In [53]:
end = int(datetime.datetime.today().timestamp())
params = {
    'period1': 728262000,
    'period2': end,
    'interval': '1d',
    'events': 'history',
    'crumb': crumb,
}
url='https://query1.finance.yahoo.com/v7/finance/download/{}'.format(ticker)

In [54]:
a = requests.get(url, params=params, headers=headers, cookies=r.cookies.get_dict())

In [55]:
a.status_code

200

In [56]:
a.url

'https://query1.finance.yahoo.com/v7/finance/download/AAPL?interval=1d&crumb=bcBshTq.f33&period2=1545064969&events=history&period1=728262000'

In [57]:
a.headers

{'X-Yahoo-Request-Id': 'bqgtohhe1fkga', 'Cache-Control': 'no-cache, no-store, private', 'X-Content-Type-Options': 'nosniff', 'Referrer-Policy': 'no-referrer-when-downgrade', 'Content-Encoding': 'gzip', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'Connection': 'keep-alive', 'Via': 'http/1.1 media-router-omega40.prod.media.ir2.yahoo.com (ApacheTrafficServer [cMsSf ]), http/1.1 media-ncache-api13.prod.media.ir2.yahoo.com (ApacheTrafficServer [cMsSf ]), http/1.1 media-ncache-api17.prod.media.ir2.yahoo.com (ApacheTrafficServer [cMsSf ]), http/1.1 media-router-api6.prod.media.ir2.yahoo.com (ApacheTrafficServer [cMsSf ]), https/1.1 e3.ycpi.waa.yahoo.com (ApacheTrafficServer [cMsSf ])', 'Strict-Transport-Security': 'max-age=15552000', 'Expect-CT': 'max-age=31536000, report-uri="http://csp.yahoo.com/beacon/csp?src=yahoocom-expect-ct-report-only"', 'Date': 'Mon, 17 Dec 2018 16:42:51 GMT', 'Content-Disposition': 'attachment; filename=AAPL.csv', 'Public-Key-Pins-Report-On

In [58]:
a.headers['Content-Disposition'].split('=')[-1]

'AAPL.csv'

In [59]:
a.content

b'Date,Open,High,Low,Close,Adj Close,Volume\n1993-01-28,2.142857,2.151786,2.116071,2.138393,0.475505,46009600\n1993-01-29,2.151786,2.187500,2.107143,2.125000,0.472527,66525200\n1993-02-01,2.116071,2.187500,2.116071,2.187500,0.486425,60138400\n1993-02-02,2.169643,2.196429,2.151786,2.151786,0.478484,45584000\n1993-02-03,2.178571,2.178571,2.089286,2.142857,0.476498,66046400\n1993-02-04,2.142857,2.151786,2.107143,2.125000,0.472527,52038000\n1993-02-05,2.116071,2.125000,2.008929,2.044643,0.454659,91904400\n1993-02-08,2.035714,2.053571,1.982143,2.017857,0.448702,70268800\n1993-02-09,2.035714,2.049107,2.017857,2.031250,0.451680,59665200\n1993-02-10,2.035714,2.044643,1.964286,1.991071,0.442746,67071200\n1993-02-11,1.991071,2.008929,1.964286,1.968750,0.437783,42067200\n1993-02-12,1.964286,1.982143,1.919643,1.924107,0.455627,68849200\n1993-02-16,1.910714,1.910714,1.839286,1.892857,0.448227,101934000\n1993-02-17,1.901786,1.928571,1.857143,1.924107,0.455627,62395200\n1993-02-18,1.964286,1.973214,1

In [60]:
df = pd.read_csv(BytesIO(a.content), index_col=['Date'], parse_dates=['Date'])

In [61]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-28,2.142857,2.151786,2.116071,2.138393,0.475505,46009600
1993-01-29,2.151786,2.187500,2.107143,2.125000,0.472527,66525200
1993-02-01,2.116071,2.187500,2.116071,2.187500,0.486425,60138400
1993-02-02,2.169643,2.196429,2.151786,2.151786,0.478484,45584000
1993-02-03,2.178571,2.178571,2.089286,2.142857,0.476498,66046400


In [62]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-11,171.660004,171.789993,167.000000,168.630005,168.630005,47281700
2018-12-12,170.399994,171.919998,169.020004,169.100006,169.100006,35627700
2018-12-13,170.490005,172.570007,169.550003,170.949997,170.949997,31898600
2018-12-14,169.000000,169.080002,165.279999,165.479996,165.479996,40634300
2018-12-17,165.449997,168.339996,165.339996,167.440994,167.440994,14806906
